### Imports

In [1]:
import pandas as pd
import requests

### Parse JSON

In [2]:
#You will need to use your own path + download from Slack, this needs to be updated to take in Christina and Orgil's work. 
#For now, we will just do local. 

PATH = '/Users/joshuadarcy/Desktop/'

In [3]:
# this function uses Python native language to load the json file named "data"

import json

with open(PATH+'restaurant_data.json') as json_file:
    data = json.load(json_file)

### Function to call USDA for single item nutrition information (Currently -> cal, sugar, fat, carbs)

In [4]:
#searching the USDA database based on an individual ingredient.
#this function is used below. 

def nutri_search(menu_item):

    #Set search parameters -- find the closest USDA entry that is similar to our ingredient. 
    
    parameters1 = {
        'api_key' : 'VaKxcy2ekEwepNcpBmsjNVLQanyrl7WwdacDS5B7',
        'q' : '{}'.format(menu_item)
    }
    
    # API call to file the ndbno (an index number that the USDA uses for nutrition information)

    response = requests.get('https://api.nal.usda.gov/ndb/search/?format=json', params = parameters1)
    JSON = response.json()
    ndbno = JSON['list']['item'][0]['ndbno']

    #Second set of parameters to call for nutrition information with our new ndbno
    
    parameters2 = {
        'api_key' : 'VaKxcy2ekEwepNcpBmsjNVLQanyrl7WwdacDS5B7',
        'nutrients':["204","205","208","269"], 
        'ndbno' : '{}'.format(ndbno)
    }

    # Second API call to get nutrition information. In order, we're getting calories, sugars, fats, and carbs
    # !!! Let me know if you want other parameters. Options can be found in the USDA download (not the API) under
    # nutrition.csv file
    
    response = requests.get('https://api.nal.usda.gov/ndb/nutrients/?format=json', params = parameters2)
    JSON = response.json()

    cal = JSON['report']['foods'][0]['nutrients'][0]['value']
    sugars = JSON['report']['foods'][0]['nutrients'][1]['value']
    fats = JSON['report']['foods'][0]['nutrients'][2]['value']
    carbs = JSON['report']['foods'][0]['nutrients'][3]['value']
    
    return cal, sugars, fats, carbs

### Search and return all estimated cal, sugars, fats, and carbs for each menu item given retaurant name. 

In [7]:
#First let's select which restaurant we want from the JSON file. Here I chose the first one in the JSON file (0)

restaurant = data['Restaurants'][0]#<--- change this number for different restaurants, right now 0 or 1

#Second we run it through a function that converts it to a python dictionary with nutrition info

restaurant_dictionary_with_nutrition = create_restaurant_dictionary_from_JSON(restaurant)

In [6]:
def create_restaurant_dictionary_from_JSON(restaurant): 
    
    #take menu from JSON file
    menu = restaurant['Menu']
    #convert to Python-friendly dictionary
    rest_dict= dict(menu)
    #for each menu item, take an ingredient, call USDA for nutrition info, and add info to dictionary
    for menu_item in list(rest_dict.keys()): 
        ingredients = rest_dict[menu_item]['ingredients']
        rest_dict[menu_item]['nutrition'] = {}
        for ingredient in ingredients:
            rest_dict[menu_item]['nutrition'][ingredient] = {}
            rest_dict[menu_item]['nutrition'][ingredient] = nutri_search(ingredient)
    #return dictionary
    return rest_dict

### Nutrition information from return above -> Score it. (Sabrina) 

In [15]:
#I'll show an example of how to pull the calories, sugars, fats, and carbs from a menu item in dictionary above


#Python doesn't allow for dictionary keys to be iterated unless you turn them into a list first
#Take index 0 of the list we just created, which happens to be named ('Freddie's Egg Salad)
item = list(restaurant_dictionary_with_nutrition.keys())[2] #<--- change this number for different menu items
nutrition_info = restaurant_dictionary_with_nutrition[item]['nutrition']

#find dictionary key "nutrition" for this item:
#return is in order calories, sugars, fats, and carbs
print(item)
print(nutrition_info)

Mary Ellen’s Chicken Salad
{'Mixed greens': ('20', '1.00', '0.00', '3.99'), 'tuna': ('250', '4.00', '12.00', '16.00'), 'mayo': ('50', '0.63', '4.93', '1.27'), 'dill bread': ('28', '--', '0.00', '7.00')}


In [17]:
nutri_search('coffee')

('10', '--', '0.00', '3.00')